In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [ ]:
file_array = next(os.walk("../resources/test_csv/"))[2]
file_array.sort()

In [ ]:
model = tf.keras.models.load_model("/home/thomashm88/other_model_speech_recognition.h5")

In [ ]:
def import_data(file_name):
    path = "../resources/test_csv/"
    test_df = pd.read_csv(os.path.join(path, file_name))
    fnames = test_df["fname"]
    test_df.drop(labels="fname", axis='columns', inplace=True)
    features = test_df.values.reshape(-1, 129, 71, 1)
    return (features, fnames)

In [ ]:
def submit(features, fnames, file_name=None):
    #classes = {0: "yes", 1: "no", 2: "up", 3: "down", 4: "left", 5: "right", 6: "on",
               #7: "off", 8: "stop", 9: "go", 10: "silence", 11: "unknown"}
    classes = {0 : "silence", 1 : "down", 2 : "go", 3 : "left", 4 : "no", 5 : "off", 6 : "on",
               7 : "right", 8 : "stop", 9 : "up", 10 : "yes", 11: "unknown"}
    preds = model.predict(features, batch_size=256, verbose=1)
    max_ind = np.argmax(preds, axis=1)
    max_val = preds.max(axis=1)
    predictions = np.where(max_val < 0.65, 11, max_ind)
    predictions = np.vectorize(classes.get)(predictions)
    submission = pd.DataFrame({"fname": fnames, "label": predictions})
    if (file_name == None):
        submission.to_csv(path_or_buf="submissions/other_submission.csv", index=False)
    else:
        submission.to_csv(path_or_buf=file_name, header=False, mode='a', index=False)
    return (submission)

In [ ]:
%%time
for i in range(8):
    print("importing data... {} out of {}".format(i + 1, 8))
    features, fnames = import_data(file_array[i])
    print("finished importing")
    if (i == 0):
        submit(features, fnames)
    else:
        submit(features, fnames, file_name="submissions/other_submission.csv")
    %reset_selective -f features
    %reset_selective -f fnames